In [18]:
import pandas as pd
import numpy as np
import math
import datetime
import tqdm as tqdm

In [19]:
cases = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_cases.csv')

cases.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3/customer_cases.csv', index=False, encoding='utf-8', sep=',')
print(len(cases))
print(cases['reason'].unique())
print(cases['channel'].unique())
cases.sort_values(by='date_time')

1330512
['signup' 'support']
['phone' 'email']


,case_id,date_time,customer_id,channel,reason
1322454,CC1322555,2017-01-01 00:03:31,C1503322,phone,signup
1076374,CC1076475,2017-01-01 00:13:11,C1257242,phone,support
1283914,CC1284015,2017-01-01 00:19:49,C1464782,phone,support
648303,CC648404,2017-01-01 01:06:33,C829171,phone,signup
820055,CC820156,2017-01-01 01:09:37,C1000923,phone,support
...,...,...,...,...,...
336702,CC336803,2021-12-31 23:24:48,C517570,email,signup
1079652,CC1079753,2021-12-31 23:39:12,C1260520,phone,signup
425562,CC425663,2021-12-31 23:44:22,C606430,phone,signup
450351,CC450452,2021-12-31 23:54:47,C631219,phone,support


In [20]:
dim_geo = pd.read_excel(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\dim_geo.xlsx')
dim_geo.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_geo.csv', sep=',', encoding='utf-8', index=False)
dim_geo.head()

,geo_id,Kommune,Region
0,1,Albertslund,Region Hovedstaden
1,2,Allerød,Region Hovedstaden
2,3,Ballerup,Region Hovedstaden
3,4,Bornholm,Region Hovedstaden
4,5,Brøndby,Region Hovedstaden


In [21]:
dim_customer = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_info.csv')

# Add age group
groups = [10,20,30,40,50,60,70,80,90]
dim_customer['age_group'] = pd.cut(dim_customer['age'], bins=groups, right=True).astype(str).str.replace(', ','-', regex=True)

# Add geography
dim_customer['geo_id'] = pd.Series(np.random.randint(1, 98, size=len(dim_customer))).astype('Int64')


dim_customer.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_customer.csv', sep=',', index=False, encoding='utf-8', )
print(len(dim_customer))
dim_customer.head()

1508932


,customer_id,age,gender,age_group,geo_id
0,C2448,76,female,(70-80],79
1,C2449,61,male,(60-70],33
2,C2450,58,female,(50-60],18
3,C2451,62,female,(60-70],60
4,C2452,71,male,(70-80],24


In [22]:
male = dim_customer.iloc[np.where(dim_customer['gender']== 'male')].customer_id.copy()
male

1             C2449
4             C2452
5             C2453
6             C2454
9             C2457
             ...   
1508927    C1511375
1508928    C1511376
1508929    C1511377
1508930    C1511378
1508931    C1511379
Name: customer_id, Length: 919500, dtype: object

In [23]:
product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_product.csv')
product['cancel_date_time'] = product['cancel_date_time'].replace(np.nan, '1/1/2100').astype(np.datetime64).copy()
product['signup_date_time'] = product['signup_date_time'].astype(np.datetime64).copy()
product['signup_month'] = product['signup_date_time'].dt.to_period('M').copy()
product['cancel_month'] = product['cancel_date_time'].dt.to_period('M').copy()


# Overwrite to make old guys use product 1
old = dim_customer.iloc[np.where(dim_customer['age']>60)].customer_id.copy()
old_lst = old.sample(100000, random_state=741).to_numpy()
product['product'][product['customer_id'].isin(old_lst)] = 'prd_1'


# Overwrite to make young guys use product 2
young = dim_customer.iloc[np.where(dim_customer['age']<45)].customer_id.copy()
young_lst = young.sample(22000, random_state=741).to_numpy()
product['product'][product['customer_id'].isin(young_lst)] = 'prd_2'


# Overwrite to make Male use product 1
male = dim_customer.iloc[np.where(dim_customer['gender']== 'male')].customer_id.copy()
male_lst = male.sample(100000, random_state=741).to_numpy()
product['product'][product['customer_id'].isin(male_lst)] = 'prd_1'



product[['customer_id', 'product', 'signup_date_time', 'cancel_date_time']].to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_product.csv', sep=',', index=False, encoding='utf-8')
print(len(product))
product.head()

C:\Users\MartinBirkAndreasen\AppData\Local\Temp\ipykernel_22840\2101287622.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['product'][product['customer_id'].isin(old_lst)] = 'prd_1'
C:\Users\MartinBirkAndreasen\AppData\Local\Temp\ipykernel_22840\2101287622.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product['product'][product['customer_id'].isin(young_lst)] = 'prd_2'
C:\Users\MartinBirkAndreasen\AppData\Local\Temp\ipykernel_22840\2101287622.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

1508932


,customer_id,product,signup_date_time,cancel_date_time,signup_month,cancel_month
0,C2448,prd_1,2017-01-01 10:35:09,2100-01-01 00:00:00,2017-01,2100-01
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,2017-01,2021-09
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,2017-01,2019-01
3,C2451,prd_2,2017-01-01 13:32:08,2100-01-01 00:00:00,2017-01,2100-01
4,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,2017-01,2021-06


In [24]:
graf = product.set_index('customer_id').join(dim_customer.set_index('customer_id'))
graf.groupby(['age_group', 'product', 'gender'])['age'].count()

age_group  product  gender
(20-30]    prd_1    female       735
                    male        1192
           prd_2    female       884
                    male        1089
(30-40]    prd_1    female      6236
                    male       10155
           prd_2    female      7394
                    male        9514
(40-50]    prd_1    female     49108
                    male       81441
           prd_2    female     33406
                    male       44590
(50-60]    prd_1    female    153130
                    male      255460
           prd_2    female     86097
                    male      119720
(60-70]    prd_1    female    151869
                    male      252394
           prd_2    female     66648
                    male       93063
(70-80]    prd_1    female     23329
                    male       37005
           prd_2    female     10596
                    male       13877
Name: age, dtype: int64

In [25]:
def num_active_user_in_a_period(start_date):
    '''count active users'''
    number_of_active =  sum((product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date))
    return number_of_active


def who_is_active_in_period(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date)
    return product[active_users]['customer_id'].to_numpy()


def who_exists(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) 
    return product[active_users]['customer_id'].reset_index(drop=True).to_numpy()


In [26]:
fact_active = pd.DataFrame()
date_list = pd.date_range(start='1/1/2017', end='1/1/2022').to_period('M').unique().to_numpy()

# List of customers who signed up and caceled by month
signup = product.groupby(['signup_month'])['customer_id'].apply(list)
cancel = product.groupby(['cancel_month'])['customer_id'].apply(list)   


# Generate fact table
for d in tqdm.tqdm(date_list[:-1]):
    
    # Get users who have been created at time d
    _users =  who_exists(d)

    # store date d, as np.array
    dates = np.repeat(d, len(_users))

    # Zip/Stack dates and users
    _df = pd.DataFrame(np.dstack((dates,_users))[0], columns = ['date','customer_id']).set_index('date')

    # is_active column
    _df['active'] = _df['customer_id'].isin(who_is_active_in_period(d))

    # new customer this month
    _df['new_customer'] = _df['customer_id'].isin(signup[d])

    # custumers who canceled
    _df['cancel'] = _df['customer_id'].isin(cancel[d])

    # Append these users 
    fact_active = pd.concat([fact_active,_df], ignore_index=False)


fact_active.index = fact_active.index.strftime('%Y-%m-%d')
print(fact_active)
fact_active.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active.csv', encoding='utf-8', sep=',', index=True)

100%|██████████| 60/60 [00:47<00:00,  1.26it/s]


           customer_id  active  new_customer  cancel
date                                                
2017-01-31       C2448    True          True   False
2017-01-31       C2449    True          True   False
2017-01-31       C2450    True          True   False
2017-01-31       C2451    True          True   False
2017-01-31       C2452    True          True   False
...                ...     ...           ...     ...
2021-12-31    C1511375   False         False   False
2021-12-31    C1511376   False         False   False
2021-12-31    C1511377   False         False   False
2021-12-31    C1511378   False         False   False
2021-12-31    C1511379    True         False   False

[32807048 rows x 4 columns]


In [27]:
dim_product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\product_info.csv')
dim_product.head()

,product_id,name,price,billing_cycle
0,prd_1,annual_subscription,1200,12
1,prd_2,monthly_subscription,125,1


# ------------  STOP HERE --------------------

# Split active users to years...

In [ ]:
fact_active['year'] = pd.to_datetime(fact_active.reset_index()['date']).dt.year.to_numpy()
fact_active2017 = fact_active[fact_active['year']==2017]
fact_active2017 = fact_active2017[['customer_id', 'active', 'new_customer', 'cancel']]
print(fact_active2017)
fact_active2017.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2017.csv', encoding='utf-8', sep=',', index=True)

           customer_id  active  new_customer  cancel
date                                                
2017-01-31       C2448    True          True   False
2017-01-31       C2449    True          True   False
2017-01-31       C2450    True          True   False
2017-01-31       C2451    True          True   False
2017-01-31       C2452    True          True   False
...                ...     ...           ...     ...
2017-12-31    C1511264    True         False   False
2017-12-31    C1511266    True          True   False
2017-12-31    C1511280    True         False   False
2017-12-31    C1511343    True         False    True
2017-12-31    C1511378    True         False   False

[466935 rows x 4 columns]


In [ ]:
fact_active2018 = fact_active[fact_active['year']==2018]
fact_active2018 = fact_active2018[['customer_id', 'active', 'new_customer', 'cancel']]
print(fact_active2018)
fact_active2018.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2018.csv', encoding='utf-8', sep=',', index=True)

           customer_id  active  new_customer  cancel
date                                                
2018-01-31       C2448    True         False   False
2018-01-31       C2449    True         False   False
2018-01-31       C2450    True         False   False
2018-01-31       C2451    True         False   False
2018-01-31       C2452    True         False   False
...                ...     ...           ...     ...
2018-12-31    C1511343   False         False   False
2018-12-31    C1511349    True          True   False
2018-12-31    C1511350    True         False    True
2018-12-31    C1511357    True         False    True
2018-12-31    C1511378    True         False   False

[1876752 rows x 4 columns]


In [ ]:
fact_active['half-year'] = np.where(pd.to_datetime(fact_active.reset_index()['date']).dt.month.le(6), 'H1', 'H2')
fact_active['quarter'] = pd.to_datetime(fact_active.reset_index()['date']).dt.to_period('Q').to_numpy()
fact_active

KeyboardInterrupt: 

In [ ]:
fact_active2019 = fact_active[fact_active['year']==2019]

fact_active2019H1 = fact_active2019[fact_active2019['half-year']=='H1']
fact_active2019H2 = fact_active2019[fact_active2019['half-year']=='H2']

fact_active2019H1 = fact_active2019H1[['customer_id', 'active', 'new_customer', 'cancel']]
fact_active2019H2 = fact_active2019H2[['customer_id', 'active', 'new_customer', 'cancel']]

print(fact_active2019H1)
print(fact_active2019H2)

fact_active2019H1.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2019H1.csv', encoding='utf-8', sep=',', index=True)
fact_active2019H2.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2019H2.csv', encoding='utf-8', sep=',', index=True)

           customer_id  active  new_customer  cancel
date                                                
2019-01-31       C2448    True         False   False
2019-01-31       C2449    True         False   False
2019-01-31       C2450    True         False    True
2019-01-31       C2451    True         False   False
2019-01-31       C2452    True         False   False
...                ...     ...           ...     ...
2019-06-30     C611370    True         False   False
2019-06-30     C611371   False         False   False
2019-06-30     C611372   False         False   False
2019-06-30     C611374   False         False   False
2019-06-30     C611377    True         False    True

[883322 rows x 4 columns]
           customer_id  active  new_customer  cancel
date                                                
2019-07-31       C2448    True         False   False
2019-07-31       C2449    True         False   False
2019-07-31       C2450   False         False   False
2019-07-31       C2

In [21]:
fact_active2020 = fact_active[fact_active['year']==2020]

use_cols = ['customer_id', 'active', 'new_customer', 'cancel']

fact_active2020Q1 = fact_active2020[fact_active2020['quarter']=='2020Q1'][use_cols]
fact_active2020Q2 = fact_active2020[fact_active2020['quarter']=='2020Q2'][use_cols]
fact_active2020Q3 = fact_active2020[fact_active2020['quarter']=='2020Q3'][use_cols]
fact_active2020Q4 = fact_active2020[fact_active2020['quarter']=='2020Q4'][use_cols]

#fact_active2020Q1 = fact_active2020Q1[['customer_id', 'active', 'new_customer', 'cancel']]
#fact_active2020H2 = fact_active2020H2[['customer_id', 'active', 'new_customer', 'cancel']]

print(fact_active2020Q2)

fact_active2020Q1.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2020Q1.csv', encoding='utf-8', sep=',', index=True)
fact_active2020Q2.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2020Q2.csv', encoding='utf-8', sep=',', index=True)
fact_active2020Q3.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2020Q3.csv', encoding='utf-8', sep=',', index=True)
fact_active2020Q4.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2020Q4.csv', encoding='utf-8', sep=',', index=True)

           customer_id  active  new_customer  cancel
date                                                
2020-04-30       C2448    True         False   False
2020-04-30       C2449    True         False   False
2020-04-30       C2450   False         False   False
2020-04-30       C2451    True         False   False
2020-04-30       C2452    True         False   False
...                ...     ...           ...     ...
2020-06-30     C611371   False         False   False
2020-06-30     C611372   False         False   False
2020-06-30     C611374   False         False   False
2020-06-30     C611375    True         False   False
2020-06-30     C611377   False         False   False

[948976 rows x 4 columns]


In [22]:
fact_active2021 = fact_active[fact_active['year']==2021]

use_cols = ['customer_id', 'active', 'new_customer', 'cancel']

fact_active2021Q1 = fact_active2021[fact_active2021['quarter']=='2021Q1'][use_cols]
fact_active2021Q2 = fact_active2021[fact_active2021['quarter']=='2021Q2'][use_cols]
fact_active2021Q3 = fact_active2021[fact_active2021['quarter']=='2021Q3'][use_cols]
fact_active2021Q4 = fact_active2021[fact_active2021['quarter']=='2021Q4'][use_cols]


print(fact_active2021Q2)

fact_active2021Q1.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021Q1.csv', encoding='utf-8', sep=',', index=True)
fact_active2021Q2.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021Q2.csv', encoding='utf-8', sep=',', index=True)
fact_active2021Q3.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021Q3.csv', encoding='utf-8', sep=',', index=True)
fact_active2021Q4.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021Q4.csv', encoding='utf-8', sep=',', index=True)

           customer_id  active  new_customer  cancel
date                                                
2021-04-30       C2448    True         False   False
2021-04-30       C2449    True         False   False
2021-04-30       C2450   False         False   False
2021-04-30       C2451    True         False   False
2021-04-30       C2452    True         False   False
...                ...     ...           ...     ...
2021-06-30     C611374   False         False   False
2021-06-30     C611375   False         False   False
2021-06-30     C611377   False         False   False
2021-06-30     C611378    True         False   False
2021-06-30     C611379    True         False   False

[1509532 rows x 4 columns]


In [24]:
fact_active2021Q3['month'] = pd.to_datetime(fact_active2021Q3.reset_index()['date']).dt.to_period('M').to_numpy()
fact_active2021M07 = fact_active2021Q3[fact_active2021Q3['month']=='2021-07'][use_cols]
fact_active2021M08 = fact_active2021Q3[fact_active2021Q3['month']=='2021-08'][use_cols]
fact_active2021M09 = fact_active2021Q3[fact_active2021Q3['month']=='2021-09'][use_cols]

fact_active2021M07.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M07.csv', encoding='utf-8', sep=',', index=True)
fact_active2021M08.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M08.csv', encoding='utf-8', sep=',', index=True)
fact_active2021M09.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M09.csv', encoding='utf-8', sep=',', index=True)

In [27]:
fact_active2021M09

,customer_id,active,new_customer,cancel
date,,,,
2021-09-30,C2448,True,False,False
2021-09-30,C2449,True,False,True
2021-09-30,C2450,False,False,False
2021-09-30,C2451,True,False,False
2021-09-30,C2452,False,False,False
...,...,...,...,...
2021-09-30,C611374,False,False,False
2021-09-30,C611375,False,False,False
2021-09-30,C611377,False,False,False


In [23]:
fact_active2021Q4['month'] = pd.to_datetime(fact_active2021Q4.reset_index()['date']).dt.to_period('M').to_numpy()
fact_active2021M10 = fact_active2021Q4[fact_active2021Q4['month']=='2021-10'][use_cols]
fact_active2021M11 = fact_active2021Q4[fact_active2021Q4['month']=='2021-11'][use_cols]
fact_active2021M12 = fact_active2021Q4[fact_active2021Q4['month']=='2021-12'][use_cols]

fact_active2021M10.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M10.csv', encoding='utf-8', sep=',', index=True)
fact_active2021M11.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M11.csv', encoding='utf-8', sep=',', index=True)
fact_active2021M12.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active2021M12.csv', encoding='utf-8', sep=',', index=True)